In [1]:
import warnings
import pprint
import logging
from collections import namedtuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
import optuna

%matplotlib inline

logging.getLogger("lightgbm").setLevel(logging.DEBUG)   # turn off lightgbm messages

In [2]:
# load data
X, y = load_breast_cancer(return_X_y=True)

# # convert y to binary (predict 2 as the positive target)
# y = np.where(y == 1, 0, y)
# y = np.where(y == 2, 1, y)

In [3]:
# split data for early stopping demonstration
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=17,
    stratify=y,
)

In [4]:
# create param space
STATIC_PARAMS = {
    "boosting_type": "goss",
    "objective": "binary",
    "random_state": 13,
    "subsample_freq": 1,
    "metric": ["auc", "binary_logloss"],
    "first_metric_only": True,
    "num_boost_round": 200,
    "early_stopping_rounds": 10,
    "verbose": -1,   # turns off lgb logging messages durng BHO
    "n_jobs": None,
}

In [5]:
# define objective function to be minimize (can also choose maximized) for optuna
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

def objective(trial):
    # suggested params for trial
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 2, 16),
        "max_depth": trial.suggest_int("max_depth", 1, 5),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.0001, 0.1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
    }
    params.update(**STATIC_PARAMS)
    
    results_dict = dict()
    model = lgb.train(
        params,
        train_data,
        valid_sets=test_data,
        evals_result=results_dict,
        verbose_eval=False,
    )
    return -results_dict["valid_0"]["auc"][-1]

In [6]:
# create study and train - note, can choose different types of hyperparameter optimization here
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=50)

[I 2022-03-01 11:38:46,592] A new study created in memory with name: no-name-7db7a67a-cf00-4bee-a5ef-a7a01a46a943
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argumen

[I 2022-03-01 11:38:46,891] Trial 6 finished with value: -0.9869304906542056 and parameters: {'num_leaves': 6, 'max_depth': 2, 'learning_rate': 0.05580834148417595, 'colsample_bytree': 0.2768464061695515, 'min_child_weight': 4, 'extra_trees': True}. Best is trial 1 with value: -0.9912383177570093.
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/li

/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in a future release of LightG

/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in a future release of LightG

[I 2022-03-01 11:38:48,194] Trial 26 finished with value: -0.9718165887850467 and parameters: {'num_leaves': 13, 'max_depth': 2, 'learning_rate': 0.0010155321561135244, 'colsample_bytree': 0.9159380514384292, 'min_child_weight': 17, 'extra_trees': False}. Best is trial 1 with value: -0.9912383177570093.
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packa

/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in a future release of LightG

[I 2022-03-01 11:38:49,073] Trial 39 finished with value: -0.9872225467289719 and parameters: {'num_leaves': 9, 'max_depth': 1, 'learning_rate': 0.0007443600863611142, 'colsample_bytree': 0.7626841621245805, 'min_child_weight': 11, 'extra_trees': True}. Best is trial 32 with value: -0.991311331775701.
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-package

/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/benjamincross/miniconda3/envs/skopt/lib/python3.8/site-packages/lightgbm/engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is deprecated and will be removed in a future release of LightG

In [7]:
study.best_params

{'num_leaves': 10,
 'max_depth': 1,
 'learning_rate': 0.00014610484805477867,
 'colsample_bytree': 0.999727693942414,
 'min_child_weight': 12,
 'extra_trees': True}

In [8]:
study.best_trial

FrozenTrial(number=44, values=[-0.991822429906542], datetime_start=datetime.datetime(2022, 3, 1, 11, 38, 49, 351874), datetime_complete=datetime.datetime(2022, 3, 1, 11, 38, 49, 406772), params={'num_leaves': 10, 'max_depth': 1, 'learning_rate': 0.00014610484805477867, 'colsample_bytree': 0.999727693942414, 'min_child_weight': 12, 'extra_trees': True}, distributions={'num_leaves': IntUniformDistribution(high=16, low=2, step=1), 'max_depth': IntUniformDistribution(high=5, low=1, step=1), 'learning_rate': LogUniformDistribution(high=0.1, low=0.0001), 'colsample_bytree': UniformDistribution(high=1.0, low=0.1), 'min_child_weight': IntUniformDistribution(high=20, low=1, step=1), 'extra_trees': CategoricalDistribution(choices=(True, False))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=44, state=TrialState.COMPLETE, value=None)

In [9]:
# from stack overflow for gpus

# differing script files (optimize.py)
# at top of script after imports
# DEVICE = sys.argv[1]    # "cuda: " + sys.argv[1] for pytorch
# def objective:
#     ...

# if __name__ == "__main__":
#     study = optuna.load_study(study_name="distributed-example", storage="sqlite:///example.db")
#.    study.optimize(objective, n_trials=100)

# in terminal
# optuna create-study --study-name "distributed-example" --storage "sqlite:///example.db"

# then for every gpu
# python optimize.py 0
# python optimize.py 1
# etc.

# see optuna docs for visualizations

# also see use of dask in github issues with shared queue (or try multiprocessing) to manage gpu ids